In [1]:
import os
%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python
hdfs_endpoint=os.environ['HDFS_ENDPOINT']

env: HDFS_PATH=/user/dominospark/small_data
env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python


In [2]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random
sys.path.insert(1, '/mnt/code/python/')
count=100000

dest_folder='/user/dominospark/largedata/'

!hdfs dfs -rmr '/user/dominospark/largedata/'

import generate_data_in_hdfs
args=[str(count),hdfs_endpoint,dest_folder,'local[*]']

generate_data_in_hdfs.main(args)

rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/largedata


In [3]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()


In [4]:
sparkSession

In [5]:

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])

df_load = sparkSession.read.csv(dest_folder,schema=columns)
df_load.show()
df_load.count()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 63| 94| 50|
|  1| 25| 26| 73|
|  2| 84| 84| 84|
|  3| 47| 19|  8|
|  4| 24| 31|  6|
|  5| 75| 17| 11|
|  6| 49| 38| 57|
|  7| 56| 31| 90|
|  8|100|  3|  3|
|  9| 43| 72| 34|
| 10| 18| 64| 57|
| 11| 63| 75| 80|
| 12| 82| 85| 28|
| 13| 31|  8| 42|
| 14| 20| 80|  3|
| 15| 27| 91| 86|
| 16| 55| 70| 42|
| 17| 69|  3|  5|
| 18| 65| 28| 28|
| 19| 57|  8| 69|
+---+---+---+---+
only showing top 20 rows



100000

In [6]:
sparkSession.stop()

In [7]:
!hdfs dfs -ls /user/dominospark/largedata/

Found 2 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-14 18:55 /user/dominospark/largedata/_SUCCESS
-rw-r--r--   1 dominospark dominospark    1464889 2021-08-14 18:55 /user/dominospark/largedata/part-00000-ed7f1bdf-15e6-46a0-aea3-7219f3bbde53-c000.csv
